# Embedding & Vector Store

In [1]:
import config
from pprint import pprint
from langchain_community.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_chroma import Chroma
import pandas as pd

import os

c:\Users\Playdata\OneDrive\코딩\SK\13_Langchain\project\config.py:20: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(model=embedding_model_name)


In [2]:
# setting

CHUNK_SIZE = config.chunk_size
CHUNK_OVERLAP = config.chunk_overlap

MODEL_NAME  = config.model_name
EMBEDDING_NAME = config.embedding_name

COLLECTION_NAME = config.collection_name
PERSIST_DIRECTORY = config.persist_directory


In [ ]:
# 데이터 로드 및 전처리
data = pd.read_csv(csv_path)
data.fillna("", inplace=True)  # NaN 값 처리

# 모든 데이터를 활용하도록 문서화
documents = []
for i, row in data.iterrows():
    # 텍스트 내용 (각 행 전체를 하나의 문서로 취급)
    page_content = "\n".join([f"{col}: {val}" for col, val in row.items()])
    
    # 메타데이터 생성 (필요 시 주요 컬럼만 선택 가능)
    metadata = row.to_dict()
    
    # Document 생성
    doc = Document(page_content=page_content, metadata=metadata)
    documents.append(doc)

print(f"총 {len(documents)}개의 문서가 생성되었습니다.")

In [6]:
####################################
# 데이터 불러오기 및 저장
####################################
from langchain_core.documents import Document
import pandas as pd
data = pd.read_csv('data/merged_restaurant_data.csv')

# 모든 데이터를 활용하도록 문서화

documents = []
for i, row in data.iterrows():
    # 텍스트 내용 (각 행 전체를 하나의 문서로 취급)
    page_content = "\n".join([f"{col}: {val}" for col, val in row.items()])
    
    # Document 생성
    doc = Document(page_content=page_content)
    documents.append(doc)

print(f"총 {len(documents)}개의 문서가 생성되었습니다.")


splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name=MODEL_NAME, 
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP
)
splited_docs = splitter.split_documents(documents)

print("데이터 분리 완료:", len(splited_docs), end='\n\n')


## Vector store 저장
embedding_model = OpenAIEmbeddings(
    model=EMBEDDING_NAME
)

## Persist directory 없는 경우 생성
if not os.path.exists(PERSIST_DIRECTORY):
    os.makedirs(PERSIST_DIRECTORY)

# 연결 + document 추가
vector_store = Chroma.from_documents(
    documents= splited_docs,
    embedding=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)

print("vecor_store에 splited_docs 저장완료")

총 39070개의 문서가 생성되었습니다.
데이터 분리 완료: 39074

vecor_store에 splited_docs 저장완료
